Let's see if we can make  a model to predict the buybox winner

In [15]:
from mlrepricer import schemas, helper
from mlrepricer.database import SQLite
import pandas as pd
import numpy as np

In [16]:
t = schemas.pricemonitor(SQLite)()  # tableobject
df = pd.read_sql_query(f'SELECT * FROM {t.table}', t.conn, parse_dates=[t.eventdate], index_col='ID')

In [17]:
# we discuss the filter1 in the views notebook
filter1 = df[(df.instock==1)&(df.isfeaturedmerchant==1)]
filter1 = filter1.drop(['instock', 'isfeaturedmerchant'], axis=1)
# that's not there yet
filter1 = filter1[(filter1.shipping_maxhours+filter1.shipping_minhours)<=72]

In [18]:
# normalize the feedback count to a number between 0 and 1
filter1['feedback'] = np.where(np.log(filter1.feedback+1)/10<=1,
                          np.log(filter1.feedback+1)/10, 1)

In [19]:
# normalize the feedbackpercent to a number between 0 and 1
filter1['feedbackpercent'] = (filter1.feedbackpercent/100)**2

In [20]:
# combine max and minshipping time to a number between 0 and 1
filter1['shipping_time'] = (filter1.shipping_maxhours+filter1.shipping_minhours)

In [21]:
# one hot encode shipping time, there should be only 3-10 unique times
filter1 = pd.get_dummies(filter1, prefix='shipping_time', columns=['shipping_time'])
filter1 = filter1.drop(['shipping_maxhours', 'shipping_minhours'], axis=1)

In [29]:
# linear transformation of none prime prices
filter1['price'] = np.where((filter1.isprime==0), filter1.price*0.859-0.567, filter1.price)

In [31]:
filter1 = filter1.drop(['isprime'], axis=1)

In [33]:
filter1.head().to_csv('test.csv')